# Bloque 1 · ¿Qué es Retrieval-Augmented Generation (RAG)?

Este bloque introduce los conceptos esenciales de un flujo RAG y cómo cada componente contribuye a obtener respuestas mejor informadas a partir de fuentes propias.


## ¿Por qué RAG?

Los modelos de lenguaje grandes (LLM) poseen conocimiento general, pero no siempre están actualizados ni conocen los detalles internos de una organización. Retrieval-Augmented Generation (RAG) permite enriquecer las respuestas de un modelo con información específica proveniente de una base de conocimiento curada.

En otras palabras, RAG combina dos mundos:
- **Recuperación**: localizar fragmentos relevantes en documentos propios.
- **Generación**: redactar una respuesta usando tanto la pregunta como los fragmentos recuperados.


## Componentes de un sistema RAG

1. **Fuentes de conocimiento**: documentos, páginas web, manuales o bases de datos que contienen la información confiable.
2. **Ingesta y limpieza**: procesos que normalizan, eliminan ruido y preparan el texto.
3. **Chunking**: división en fragmentos manejables (chunks) que capturen ideas completas.
4. **Modelos de embeddings**: convierten cada fragmento en un vector numérico que preserva significado.
5. **Base vectorial**: almacena los vectores y permite realizar búsquedas por similitud.
6. **Recuperador**: dada una consulta, encuentra los vectores más cercanos.
7. **Generador**: redacta la respuesta final usando la consulta y los fragmentos recuperados.


## Flujo típico de extremo a extremo

1. Se seleccionan y preparan las fuentes confiables.
2. Se crean fragmentos y sus embeddings.
3. Se almacenan en una base vectorial.
4. Una pregunta del usuario se transforma en vector.
5. Se recuperan los fragmentos más similares.
6. Un modelo genera una respuesta apoyándose en esos fragmentos.

En este taller simularemos los pasos 1 a 5 y omitiremos la generación automática para que podamos observar el efecto de la recuperación.


## Actividad práctica: simulación de vector search

Trabajaremos con un conjunto reducido de fragmentos y construiremos manualmente una representación vectorial sencilla para entender cómo cambia la recuperación cuando variamos la pregunta.


### Paso 1. Definir una base de conocimiento

Imaginemos que recopilamos notas internas sobre cómo implementar un sistema RAG. Cada elemento representa un párrafo breve que podremos fragmentar más adelante.


In [ ]:
import numpy as np
from collections import Counter

corpus = [
    {
        "id": "doc1",
        "titulo": "Arquitectura general",
        "contenido": (
            "Un sistema RAG combina recuperación de información y modelos generativos. "
            "La clave es aportar contexto actualizado y específico mediante documentos propios."
        ),
    },
    {
        "id": "doc2",
        "titulo": "Preparación de datos",
        "contenido": (
            "Antes de indexar documentos conviene limpiar el texto, eliminar duplicados "
            "y segmentar en fragmentos que mantengan sentido completo."
        ),
    },
    {
        "id": "doc3",
        "titulo": "Embeddings",
        "contenido": (
            "Los embeddings convierten texto en vectores. "
            "Dos fragmentos semánticamente similares tendrán vectores cercanos en el espacio."
        ),
    },
    {
        "id": "doc4",
        "titulo": "Vector store",
        "contenido": (
            "Una base vectorial almacena los embeddings y ofrece métricas de similitud. "
            "Es importante elegir índices que escalen con la cantidad de documentos."
        ),
    },
    {
        "id": "doc5",
        "titulo": "Buenas prácticas",
        "contenido": (
            "La calidad de un RAG depende de la curación de las fuentes, la estrategia de chunking "
            "y el ajuste de la consulta. Medir relevancia ayuda a iterar."
        ),
    },
]

print(f"Documentos disponibles: {len(corpus)}")


### Paso 2. Fragmentar y tokenizar

Dividiremos cada documento en frases cortas (chunks) y convertiremos el texto en tokens sencillos. Para mantener el enfoque introductorio, utilizaremos una tokenización muy básica basada en separar por espacios.


In [ ]:
def sentence_chunks(text):
    delimiters = ".!?"
    current = ""
    for char in text:
        current += char
        if char in delimiters:
            chunk = current.strip()
            if chunk:
                yield chunk
            current = ""
    if current.strip():
        yield current.strip()

def tokenize(text):
    table = str.maketrans({c: " " for c in ",.;:¡!¿?\"'"})
    cleaned = text.lower().translate(table)
    return [token for token in cleaned.split() if token]

chunks = []
for doc in corpus:
    for idx, fragment in enumerate(sentence_chunks(doc["contenido"])):
        tokens = tokenize(fragment)
        chunks.append(
            {
                "doc_id": doc["id"],
                "titulo": doc["titulo"],
                "fragmento": fragment,
                "tokens": tokens,
            }
        )

print(f"Chunks generados: {len(chunks)}")
print(
    f"Primer chunk de ejemplo:\n- Documento: {chunks[0]['doc_id']} ({chunks[0]['titulo']})\n- Texto: {chunks[0]['fragmento']}"
)


### Paso 3. Construir un espacio vectorial simple

Crearemos un vocabulario con todas las palabras observadas y representaremos cada chunk como un vector de frecuencias normalizadas (TF). Aunque en producción usaríamos modelos más avanzados, este enfoque permite visualizar el mecanismo de la similitud coseno.


In [ ]:
vocabulary = sorted({token for chunk in chunks for token in chunk["tokens"]})
token_index = {token: idx for idx, token in enumerate(vocabulary)}

def vectorize(tokens):
    if not tokens:
        return np.zeros(len(vocabulary), dtype=float)
    filtered = [token for token in tokens if token in token_index]
    if not filtered:
        return np.zeros(len(vocabulary), dtype=float)
    counts = Counter(filtered)
    total = sum(counts.values())
    vector = np.zeros(len(vocabulary), dtype=float)
    for token, freq in counts.items():
        idx = token_index[token]
        vector[idx] = freq / total
    return vector

chunk_vectors = np.vstack([vectorize(chunk["tokens"]) for chunk in chunks])

print(f"Tamaño del vocabulario: {len(vocabulary)} términos")


### Paso 4. Recuperar fragmentos por similitud

Compararemos la representación vectorial de la pregunta con cada chunk y ordenaremos por similitud coseno. Esto simula el comportamiento de una base vectorial.


In [ ]:
def cosine_similarity(vector_a, vector_b):
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    if denom == 0:
        return 0.0
    return float(np.dot(vector_a, vector_b) / denom)

def retrieve(query, top_k=3):
    query_tokens = tokenize(query)
    query_vector = vectorize(query_tokens)
    scores = [cosine_similarity(query_vector, chunk_vectors[idx]) for idx in range(len(chunks))]
    ranked = sorted(
        zip(range(len(chunks)), scores), key=lambda item: item[1], reverse=True
    )
    top_hits = []
    for idx, score in ranked[:top_k]:
        chunk = chunks[idx]
        top_hits.append(
            {
                "similitud": score,
                "fragmento": chunk["fragmento"],
                "titulo": chunk["titulo"],
                "doc_id": chunk["doc_id"],
            }
        )
    return top_hits

def mostrar_resultados(pregunta, top_k=3):
    print(f"Pregunta: {pregunta}\n")
    resultados = retrieve(pregunta, top_k=top_k)
    for pos, hit in enumerate(resultados, start=1):
        porcentaje = round(hit["similitud"] * 100, 2)
        print(f"#{pos} · similitud: {porcentaje}%")
        print(f"Documento: {hit['doc_id']} · {hit['titulo']}")
        print(f"Fragmento: {hit['fragmento']}")
        print("-" * 60)

ejemplo = "¿Para qué sirve la base vectorial?"
mostrar_resultados(ejemplo)


### Experimenta cambiando la pregunta

Ejecuta la celda siguiente y modifica el texto de la pregunta para observar cómo cambian los fragmentos recuperados. Prueba preguntas sobre "embeddings", "preparación", "buenas prácticas" u otros temas mencionados en la base de conocimiento.


In [ ]:
otra_pregunta = "¿Qué pasos recomiendan para preparar los datos?"
mostrar_resultados(otra_pregunta)


### Reflexión

- Los resultados dependen de las palabras que comparten la pregunta y los fragmentos.
- En sistemas reales usaríamos embeddings semánticos más robustos y normalizaciones adicionales.
- Una vez recuperados los fragmentos, el siguiente paso sería pasarlos a un modelo generativo para redactar una respuesta final.

En el siguiente bloque profundizaremos en cómo mejorar la representación vectorial y evaluaremos la calidad de la recuperación.
